# Oil Environment Code Demo

This problem, adaptved from [here](https://www.pnas.org/content/109/3/764) is a continuous variant of the “Grid World” environment. It comprises of an agent surveying a d-dimensional map in search of hidden “oil deposits”. The world is endowed with an unknown survey function which encodes the probability of observing oil at that specific location. For agents to move to a new location they pay a cost proportional to the distance moved, and surveying the land produces noisy estimates of the true value of that location. In addition, due to varying terrain the true location the agent moves to is perturbed as a function of the state and action.


There is a $d$-dimensional reinforcement learning environment in the space $X = [0, 1]^d$.  The action space $A = [0,1]^d$ corresponding to the ability to attempt to move to any desired location within the state space.  On top of that, there is a corresponding reward function $f_h(x,a)$ for the reward for moving the agent to that location.  Moving also causes an additional cost $\alpha d(x,a)$ scaling with respect to the distance moved.

Here is an example of the oil discovery problem 

In this notebook we run a sample experiment for the setting when $d = 1$ and the reward function is taken to be a quadratic.  We compare several heuristics to existing reinforcement learning algorithms.

### Package Installation

In [19]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

### Experiment Parameters

Here we use the oil environment as outlined in `or_suite/envs/oil_discovery/oil_environment.py`.  The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use one the defaults.

In addition, we need to specify the number of episodes for learning, and the number of iterations (in order to plot average results with confidence intervals).

In [20]:
CONFIG =  or_suite.envs.env_configs.oil_environment_default_config

epLen = CONFIG['epLen']
nEps = 50
numIters = 2

epsilon = (nEps * epLen)**(-1 / 4)
action_net = np.arange(start=0, stop=1, step=epsilon)
state_net = np.arange(start=0, stop=1, step=epsilon)

scaling_list = [0.1, 0.3, 1, 5]

DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/oil/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }



### Specifying Agent

We specify 6 different agents to compare the effectiveness of each.

* `SB PPO` is Proximal Policy Optimization. When policy is updated, there is a parameter that “clips” each policy update so that action update does not go too far
* `Random` implements the randomized RL algorithm, which selects an action uniformly at random from the action space. In particular, the algorithm stores an internal copy of the environment’s action space and samples uniformly at random from it.
* `AdaQL` is an Adaptive Discretization Model-Free Agent, implemented for enviroments with continuous states and actions using the metric induced by the l_inf norm.
* `AdaMB` is an Adaptive Discretizaiton Model-Based Agent, implemented for enviroments with continuous states and actions using the metric induced by the l_inf norm.
* `Unif QL` is an eNet Model-Based Agent, implemented for enviroments with continuous states and actions using the metric induces by the l_inf norm.
* `Unif MB` is a eNet Model-Free Agent, implemented for enviroments with continuous states and actions using the metric induces by the l_inf norm.

In [23]:
oil_env = gym.make('Oil-v0', config=CONFIG)
mon_env = Monitor(oil_env)
dim = CONFIG['dim']
cost_param = CONFIG['cost_param']
prob = CONFIG['oil_prob']

agents = { 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'AdaQL': or_suite.agents.rl.ada_ql.AdaptiveDiscretizationQL(epLen, scaling_list[0], True, dim*2),
'AdaMB': or_suite.agents.rl.ada_mb.AdaptiveDiscretizationMB(epLen, scaling_list[0], 0, 2, True, True, dim, dim),
'Unif QL': or_suite.agents.rl.enet_ql.eNetQL(action_net, state_net, epLen, scaling_list[0], (dim,dim)),
'Unif MB': or_suite.agents.rl.enet_mb.eNetMB(action_net, state_net, epLen, scaling_list[0], (dim,dim), 0, False),
}

/Users/jren/opt/anaconda3/envs/ORSuite/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:131: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a multiple of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  warnings.warn(


### Running Algorithm

In [24]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(oil_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(oil_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
        algo_list_radar.append(str(agent))

fig_path = '../figures/'
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

SB PPO
New Experiment Run
Iteration: 0
Iteration: 1
[3.724073352353859, 3.277192355871049, 3.0170077085104774, 3.616607795859035, 4.321952176285109, 3.862295333499586, 3.71105293282902, 4.096212791997065, 3.0865772805106735, 5.0, 3.1743407714222234, 2.5152341901044637, 3.648680062153219, 2.9370053265445293, 4.333806080489662, 5.0, 3.601817322045906, 3.7034196232366794, 4.246525126945727, 2.9235048752929322, 3.7357588823428847, 3.1123353322604568, 3.0275922007479616, 3.7357588823428847, 3.8537478455985426, 3.065924264880546, 3.2070980711453783, 3.9892367417912857, 4.60716227535491, 3.7077338111062668, 4.279270616763453, 3.193327939287183, 4.3071012335381305, 5.0, 4.797579265703985, 5.0, 4.9138835452560325, 3.803335756227189, 4.994924321114546, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.752929232341586, 5.0, 5.0, 4.220734244555915, 5.0, 4.908306063821534, 4.367879441171443, 5.0, 3.9380751601602184, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5

{'iter': 0, 'episode': 36, 'step': 0, 'oldState': array([0]), 'action': array([0.15045229], dtype=float32), 'reward': 0.8603187782739846, 'newState': array([0.15045229]), 'info': {}}
{'iter': 0, 'episode': 36, 'step': 1, 'oldState': array([0.15045229]), 'action': array([0.8603503], dtype=float32), 'reward': 0.49169433553396547, 'newState': array([0.86035031]), 'info': {}}
{'iter': 0, 'episode': 36, 'step': 2, 'oldState': array([0.86035031]), 'action': array([0.7813542], dtype=float32), 'reward': 0.9240435079866126, 'newState': array([0.78135419]), 'info': {}}
{'iter': 0, 'episode': 36, 'step': 3, 'oldState': array([0.78135419]), 'action': array([0.20657972], dtype=float32), 'reward': 0.562831787749567, 'newState': array([0.20657972]), 'info': {}}
{'iter': 0, 'episode': 36, 'step': 4, 'oldState': array([0.20657972]), 'action': array([0.6092777], dtype=float32), 'reward': 0.6685139532339903, 'newState': array([0.60927773]), 'info': {}}
{'iter': 0, 'episode': 37, 'step': 0, 'oldState': ar

{'iter': 1, 'episode': 21, 'step': 4, 'oldState': array([0.27880442]), 'action': array([0.30338508], dtype=float32), 'reward': 0.9757189867077378, 'newState': array([0.30338508]), 'info': {}}
{'iter': 1, 'episode': 22, 'step': 0, 'oldState': array([0]), 'action': array([0.08264601], dtype=float32), 'reward': 0.9206769978703605, 'newState': array([0.08264601]), 'info': {}}
{'iter': 1, 'episode': 22, 'step': 1, 'oldState': array([0.08264601]), 'action': array([0.8128407], dtype=float32), 'reward': 0.48181517740063534, 'newState': array([0.8128407]), 'info': {}}
{'iter': 1, 'episode': 22, 'step': 2, 'oldState': array([0.8128407]), 'action': array([0.73127705], dtype=float32), 'reward': 0.9216740419945711, 'newState': array([0.73127705]), 'info': {}}
{'iter': 1, 'episode': 22, 'step': 3, 'oldState': array([0.73127705]), 'action': array([0.3610246], dtype=float32), 'reward': 0.6905599696920848, 'newState': array([0.36102459]), 'info': {}}
{'iter': 1, 'episode': 22, 'step': 4, 'oldState': ar

{'iter': 0, 'episode': 6, 'step': 4, 'oldState': array([0.86299902]), 'action': array([0.94165305]), 'reward': 0.9243596505940529, 'newState': array([0.94165307]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 0, 'oldState': array([0]), 'action': array([0.08722459]), 'reward': 0.916471244259377, 'newState': array([0.08722459]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 1, 'oldState': array([0.08722459]), 'action': array([0.06748197]), 'reward': 0.9804509937019296, 'newState': array([0.06748197]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 2, 'oldState': array([0.06748197]), 'action': array([0.48242002]), 'reward': 0.6603811867645892, 'newState': array([0.48242003]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 3, 'oldState': array([0.48242003]), 'action': array([0.83172075]), 'reward': 0.7051810207205578, 'newState': array([0.83172077]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 4, 'oldState': array([0.83172077]), 'action': array([0.97474463]), 'reward': 0.86673339832841, '

{'iter': 0, 'episode': 38, 'step': 0, 'oldState': array([0]), 'action': array([0.36310121]), 'reward': 0.6955160338580287, 'newState': array([0.36310121]), 'info': {}}
{'iter': 0, 'episode': 38, 'step': 1, 'oldState': array([0.36310121]), 'action': array([0.86850098]), 'reward': 0.6032643461134486, 'newState': array([0.86850101]), 'info': {}}
{'iter': 0, 'episode': 38, 'step': 2, 'oldState': array([0.86850101]), 'action': array([0.45079084]), 'reward': 0.6585530592919884, 'newState': array([0.45079082]), 'info': {}}
{'iter': 0, 'episode': 38, 'step': 3, 'oldState': array([0.45079082]), 'action': array([0.37963263]), 'reward': 0.9313145460437147, 'newState': array([0.37963262]), 'info': {}}
{'iter': 0, 'episode': 38, 'step': 4, 'oldState': array([0.37963262]), 'action': array([0.34673429]), 'reward': 0.967636927801415, 'newState': array([0.34673429]), 'info': {}}
{'iter': 0, 'episode': 39, 'step': 0, 'oldState': array([0]), 'action': array([0.00129553]), 'reward': 0.9987053061581984, 'n

...

{'iter': 0, 'episode': 6, 'step': 0, 'oldState': array([0]), 'action': array([0.75446006]), 'reward': 0.47026447171799, 'newState': array([0.75446004]), 'info': {}}
{'iter': 0, 'episode': 6, 'step': 1, 'oldState': array([0.75446004]), 'action': array([0.75446006]), 'reward': 1.0, 'newState': array([0.75446004]), 'info': {}}
{'iter': 0, 'episode': 6, 'step': 2, 'oldState': array([0.75446004]), 'action': array([0.25148669]), 'reward': 0.6047299115000284, 'newState': array([0.25148669]), 'info': {}}
{'iter': 0, 'episode': 6, 'step': 3, 'oldState': array([0.25148669]), 'action': array([0.50297337]), 'reward': 0.7776438088724638, 'newState': array([0.50297338]), 'info': {}}
{'iter': 0, 'episode': 6, 'step': 4, 'oldState': array([0.50297338]), 'action': array([0.50297337]), 'reward': 1.0, 'newState': array([0.50297338]), 'info': {}}
{'iter': 0, 'episode': 7, 'step': 0, 'oldState': array([0]), 'action': array([0.25148669]), 'reward': 0.7776438088724638, 'newState': array([0.25148669]), 'info'

{'iter': 0, 'episode': 30, 'step': 2, 'oldState': array([0.25148669]), 'action': array([0.50297337]), 'reward': 0.7776438088724638, 'newState': array([0.50297338]), 'info': {}}
{'iter': 0, 'episode': 30, 'step': 3, 'oldState': array([0.50297338]), 'action': array([0.50297337]), 'reward': 1.0, 'newState': array([0.50297338]), 'info': {}}
{'iter': 0, 'episode': 30, 'step': 4, 'oldState': array([0.50297338]), 'action': array([0.75446006]), 'reward': 0.7776438320480555, 'newState': array([0.75446004]), 'info': {}}
{'iter': 0, 'episode': 31, 'step': 0, 'oldState': array([0]), 'action': array([0.50297337]), 'reward': 0.6047298934776729, 'newState': array([0.50297338]), 'info': {}}
{'iter': 0, 'episode': 31, 'step': 1, 'oldState': array([0.50297338]), 'action': array([0.75446006]), 'reward': 0.7776438320480555, 'newState': array([0.75446004]), 'info': {}}
{'iter': 0, 'episode': 31, 'step': 2, 'oldState': array([0.75446004]), 'action': array([0.25148669]), 'reward': 0.6047299115000284, 'newSta

{'iter': 1, 'episode': 4, 'step': 0, 'oldState': array([0]), 'action': array([0.]), 'reward': 1.0, 'newState': array([0.]), 'info': {}}
{'iter': 1, 'episode': 4, 'step': 1, 'oldState': array([0.]), 'action': array([0.]), 'reward': 1.0, 'newState': array([0.]), 'info': {}}
{'iter': 1, 'episode': 4, 'step': 2, 'oldState': array([0.]), 'action': array([0.50297337]), 'reward': 0.6047298934776729, 'newState': array([0.50297338]), 'info': {}}
{'iter': 1, 'episode': 4, 'step': 3, 'oldState': array([0.50297338]), 'action': array([0.]), 'reward': 0.6047298934776729, 'newState': array([0.]), 'info': {}}
{'iter': 1, 'episode': 4, 'step': 4, 'oldState': array([0.]), 'action': array([0.]), 'reward': 1.0, 'newState': array([0.]), 'info': {}}
{'iter': 1, 'episode': 5, 'step': 0, 'oldState': array([0]), 'action': array([0.25148669]), 'reward': 0.7776438088724638, 'newState': array([0.25148669]), 'info': {}}
{'iter': 1, 'episode': 5, 'step': 1, 'oldState': array([0.25148669]), 'action': array([0.754460

{'iter': 1, 'episode': 27, 'step': 0, 'oldState': array([0]), 'action': array([0.75446006]), 'reward': 0.47026447171799, 'newState': array([0.75446004]), 'info': {}}
{'iter': 1, 'episode': 27, 'step': 1, 'oldState': array([0.75446004]), 'action': array([0.50297337]), 'reward': 0.7776438320480555, 'newState': array([0.50297338]), 'info': {}}
{'iter': 1, 'episode': 27, 'step': 2, 'oldState': array([0.50297338]), 'action': array([0.75446006]), 'reward': 0.7776438320480555, 'newState': array([0.75446004]), 'info': {}}
{'iter': 1, 'episode': 27, 'step': 3, 'oldState': array([0.75446004]), 'action': array([0.50297337]), 'reward': 0.7776438320480555, 'newState': array([0.50297338]), 'info': {}}
{'iter': 1, 'episode': 27, 'step': 4, 'oldState': array([0.50297338]), 'action': array([0.25148669]), 'reward': 0.7776438088724638, 'newState': array([0.25148669]), 'info': {}}
{'iter': 1, 'episode': 28, 'step': 0, 'oldState': array([0]), 'action': array([0.]), 'reward': 1.0, 'newState': array([0.]), '

0.1
  Algorithm    Reward      Time   Space
0    Random  3.665350  5.092520 -8459.5
1     AdaQL  4.528145  4.938095 -9191.5
2     AdaMB  4.725755  5.018150 -7295.0
3   Unif QL  4.290420  5.119655 -7701.5
4   Unif MB  5.000000  4.824885 -7705.0


Here we see the uniform discretization model based algorithm performs the best with a minimal time complexity for evaluating the algorithm.